In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
import ast

# Read the CSV
df = pd.read_csv('Data/SlidersTrajectoriesAustin.csv')

# Ensure time_diff is numeric
df['time_diff'] = pd.to_numeric(df['time_diff'], errors='coerce')

# Helper to parse polynomial coefficient strings like "[A, B, C]"
def _parse_poly(s):
    if pd.isna(s) or s is None:
        return None
    try:
        v = ast.literal_eval(s)
        return list(v)
    except Exception:
        s2 = str(s).strip().strip('[]')
        if s2 == '':
            return None
        return [float(x) for x in s2.split(',')]

# Prepare container and iterate rows, starting a new pitch when time_diff == 0
pitches = []
current = {
    'time_diff': [],
    'x': [],
    'y': [],
    'z': [],
    'traj_poly_x': None,
    'traj_poly_y': None,
    'traj_poly_z': None,
    'horz_break': None,
    'induced_vert_break': None
}

for _, row in df.iterrows():
    t = row['time_diff']
    # start a new pitch when time_diff == 0 and current already has data
    if t == 0 and current['time_diff']:
        pitches.append(current)
        current = {
            'time_diff': [],
            'x': [],
            'y': [],
            'z': [],
            'traj_poly_x': None,
            'traj_poly_y': None,
            'traj_poly_z': None,
            'horz_break': None,
            'induced_vert_break': None
        }

    current['time_diff'].append(float(t) if not pd.isna(t) else None)
    current['x'].append(float(row['x']) if not pd.isna(row['x']) else None)
    current['y'].append(float(row['y']) if not pd.isna(row['y']) else None)
    current['z'].append(float(row['z']) if not pd.isna(row['z']) else None)

    # For polynomial coefficients and break values, record once per pitch (first non-null encountered)
    if current['traj_poly_x'] is None:
        px = row.get('traj_poly_x', None)
        if not pd.isna(px):
            current['traj_poly_x'] = _parse_poly(px)
    if current['traj_poly_y'] is None:
        py = row.get('traj_poly_y', None)
        if not pd.isna(py):
            current['traj_poly_y'] = _parse_poly(py)
    if current['traj_poly_z'] is None:
        pz = row.get('traj_poly_z', None)
        if not pd.isna(pz):
            current['traj_poly_z'] = _parse_poly(pz)

    if current['horz_break'] is None:
        hb = row.get('horz_break', None)
        if not pd.isna(hb):
            current['horz_break'] = float(hb)
    if current['induced_vert_break'] is None:
        ivb = row.get('induced_vert_break', None)
        if not pd.isna(ivb):
            current['induced_vert_break'] = float(ivb)

# append the last pitch if present
if current['time_diff']:
    pitches.append(current)

print(f'Parsed {len(pitches)} pitches')
# `pitches` is a list of dictionaries, each with the requested keys

Parsed 77272 pitches


In [4]:
# print the last pitch from the existing `pitches` list
last_idx = len(pitches) - 1
last_pitch = pitches[last_idx]

print(f"Last pitch index: {last_idx}")

# show time-series columns as a small DataFrame slice
series_keys = ['time_diff', 'x', 'y', 'z']
df_last = pd.DataFrame({k: last_pitch[k] for k in series_keys})
print("\nTime-series preview (head):")
print(df_last.head().to_string(index=False))
print("\nTime-series preview (tail):")
print(df_last.tail().to_string(index=False))

# print trajectory polynomials and break values
print("\nTrajectory polynomials and break values:")
for k in ('traj_poly_x', 'traj_poly_y', 'traj_poly_z', 'horz_break', 'induced_vert_break'):
    print(f"{k}: {last_pitch.get(k)}")

Last pitch index: 77271

Time-series preview (head):
 time_diff         x         y        z
     0.000 -1.397562 52.817164 5.729274
     0.033 -1.295094 48.375291 5.610428
     0.033 -1.178750 43.954413 5.456601
     0.034 -1.055861 39.567517 5.268647
     0.033 -0.935027 35.215398 5.051698

Time-series preview (tail):
 time_diff         x         y        z
     0.033 -0.533544 22.306197 4.202487
     0.033 -0.393970 18.060299 3.856825
     0.034 -0.246746 13.841784 3.475933
     0.033 -0.091108  9.641038 3.062432
     0.033  0.064966  5.471616 2.608659

Trajectory polynomials and break values:
traj_poly_x: [-1.4401106821506984, 3.2164345225825404, 2.067296898895956]
traj_poly_y: [54.13313945653019, -133.8687369509712, 12.375075295555465]
traj_poly_z: [5.757673844036027, -2.8359301293098427, -14.711198414570916]
horz_break: -4.770845689
induced_vert_break: 2.307252208
